## This Document is for creating custom generator, layers, callbacks, metrics

In [2]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imgaug
from imgaug import augmenters as iaa
import cv2

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

In [5]:
print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [6]:
y_train.shape

(50000, 1)

In [7]:
len(np.unique(y_train))

100

In [0]:
class CIFAR_Generator(tf.keras.utils.Sequence):
  def __init__(self, x_data, y_data, batch_size, target_size, shuffle=True, augmentation=True):
    self.x_data = x_data
    self.y_data = y_data
    self.batch_size = batch_size
    self.target_size = target_size
    self.shuffle = shuffle
    self.augmentation = augmentation
    self.on_epoch_end()

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.x_data))
    if self.shuffle:
      np.random.shuffle(self.indexes)
  
  def __len__(self):
    return int(np.ceil(len(self.x_data) / self.batch_size))

  def __getitem__(self,  index):
    batch_index = self.indexes[index * self.batch_size : (index+1) * self.batch_size]

    x_batch, y_batch = [self.__data_generate(self.x_data, self.y_data)[k] for k in batch_index]
    
    return x_batch, y_batch

  def __img_preprocess(self,x_data):
    x_data = (x_data - 127.5) / 127.5
    # assert len(x_data.shape) == 3
    return x_data 

  def __resize_image(self, image):
    '''
    cv2 requires to use for loop
    '''
    image = cv2.resize(image, dsize=self.target_size)
    # image = np.array([cv2.resize(i, dsize=self.target_size) for i in image])
    # image = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2RGB) for i in image])
    return image

  def __data_generate(self, x_data, y_data):
    x_data = self.__resize_image(x_data)
    x_data = self.__img_preprocess(x_data)
    if self.augmentation:
      x_data = self.__augmentation__(x_data)
    y_data = tf.keras.utils.to_categorical(y_data, num_classes=100)

    return x_data, y_data

  def __augmentation__(self, x_data):
    somtimes = lambda aug : iaa.Sometimes(0.5, aug)

    seq = iaa.Sequential([
        # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.2), # vertically flip 20% of all images
        # crop images by -5% to 10% of their height/width
        sometimes(iaa.CropAndPad(
            percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        )),
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-45, 45), # rotate by -45 to +45 degrees
            shear=(-16, 16), # shear by -16 to +16 degrees
            order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
        # execute 0 to 5 of the following (less important) augmenters per image
        # don't execute all of them, as that would often be way too strong
        iaa.SomeOf((0, 5),
            [
                sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
                iaa.OneOf([
                    iaa.GaussianBlur((0, 3.0)), # blur images with a sigma between 0 and 3.0
                    iaa.AverageBlur(k=(2, 7)), # blur image using local means with kernel sizes between 2 and 7
                    iaa.MedianBlur(k=(3, 11)), # blur image using local medians with kernel sizes between 2 and 7
                ]),
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
                # search either for all edges or for directed edges,
                # blend the result with the original image using a blobby mask
                iaa.SimplexNoiseAlpha(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0.5, 1.0)),
                    iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                ])),
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
                iaa.OneOf([
                    iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
                    iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                ]),
                iaa.Invert(0.05, per_channel=True), # invert color channels
                iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                # either change the brightness of the whole image (sometimes
                # per channel) or change the brightness of subareas
                iaa.OneOf([
                    iaa.Multiply((0.5, 1.5), per_channel=0.5),
                    iaa.FrequencyNoiseAlpha(
                        exponent=(-4, 0),
                        first=iaa.Multiply((0.5, 1.5), per_channel=True),
                        second=iaa.LinearContrast((0.5, 2.0))
                    )
                ]),
                iaa.LinearContrast((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
                iaa.Grayscale(alpha=(0.0, 1.0)),
                sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
                sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
            ], random_order=True)],random_order=True)
    image_aug = seq(images=x_data)
    return image_aug



In [0]:
test_gen = CIFAR_Generator(x_train, y_train, 10, (32,32), shuffle=False, augmentation=False)

In [11]:
test_gen.indexes

array([    0,     1,     2, ..., 49997, 49998, 49999])

In [12]:
for n, i in enumerate(test_gen):
  print(i)

  if n == 0:
    break

error: ignored

In [0]:
test_gen = CIFAR_Generator(x_train, y_train, 10, shuffle=True, augmentation=False)

In [23]:
test_gen.indexes

array([17482, 41448, 46467, ..., 31272, 46831, 20414])

In [0]:
assert len(x_train[0].shape) == 3